### Package needed!

In [1]:
# !pip install flask flask_cors

### Create a database

In [2]:
import os
import sqlite3 as sqlite
def create_db(db_file): 
    if not os.path.exists(db_file):
        print('Creating schema')
        conn = sqlite.connect(db_file)
        sql = "CREATE TABLE result (url TEXT, html_bin BLOB)"
        conn.execute(sql) # shortcut for conn.cursor().execute(sql)
        conn.commit() 
        conn.close()  
    else:
        print('Schema exists\n') 
create_db('w3school.db')
import pandas
pandas.read_sql_query("SELECT * FROM result", sqlite.connect('w3school.db'))

Creating schema


,url,html_bin


### Set up a web server

In [3]:
from flask import Flask, request, make_response, jsonify 
from flask_cors import CORS  
 
app = Flask(__name__) 
CORS(app)

#### Database app

In [4]:
@app.route('/database', methods = ['POST']) 
def database():  
    url = request.form['url'] 
    doc = request.form['doc'].encode() 
    conn = sqlite.connect('w3school.db')
    c = conn.cursor()   
    c.execute('''INSERT INTO result VALUES (?, ?)''', [url, sqlite.Binary(doc)]) 
    conn.commit() 
    conn.close()  
    print(url, "") 
    return 'Data has been inserted into the Database!' 

#### URL provider

In [5]:
@app.route('/get_url') 
def get_url(): 
    return jsonify(url = [
                            'https://www.w3schools.com/jsref/obj_window.asp', 
                            'https://www.w3schools.com/jsref/prop_doc_url.asp',
                            'https://www.w3schools.com/jsref/jsref_obj_json.asp',
                            'https://www.w3schools.com/jsref/jsref_obj_error.asp',
                            'https://www.w3schools.com/jsref/jsref_obj_array.asp',
                            'https://yzlab3.chem.nyu.edu/hungarian/', 
                            'https://yzlab3.chem.nyu.edu/C11/' 
                         ])

### Run both apps

In [6]:
app.run('0.0.0.0', 9999)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
192.168.56.1 - - [16/Dec/2018 15:08:18] "GET /get_url HTTP/1.1" 200 -
192.168.56.1 - - [16/Dec/2018 15:08:18] "POST /database HTTP/1.1" 200 -


https://yzlab3.chem.nyu.edu/hungarian/ 


192.168.56.1 - - [16/Dec/2018 15:08:19] "POST /database HTTP/1.1" 200 -


https://yzlab3.chem.nyu.edu/C11/ 


192.168.56.1 - - [16/Dec/2018 15:08:21] "POST /database HTTP/1.1" 200 -
192.168.56.1 - - [16/Dec/2018 15:08:21] "POST /database HTTP/1.1" 200 -


https://www.w3schools.com/jsref/prop_doc_url.asp 
https://www.w3schools.com/jsref/jsref_obj_array.asp 


192.168.56.1 - - [16/Dec/2018 15:08:21] "POST /database HTTP/1.1" 200 -
192.168.56.1 - - [16/Dec/2018 15:08:21] "POST /database HTTP/1.1" 200 -
192.168.56.1 - - [16/Dec/2018 15:08:21] "POST /database HTTP/1.1" 200 -


https://www.w3schools.com/jsref/obj_window.asp 
https://www.w3schools.com/jsref/jsref_obj_json.asp 
https://www.w3schools.com/jsref/jsref_obj_error.asp 


### Inspect the result

In [7]:
df = pandas.read_sql_query("SELECT * FROM result", sqlite.connect('w3school.db'))
df

,url,html_bin
0,https://yzlab3.chem.nyu.edu/hungarian/,b'<html><head>\n <title>Index of /hungarian</...
1,https://yzlab3.chem.nyu.edu/C11/,b'<html><head>\n <title>Index of /C11</title>...
2,https://www.w3schools.com/jsref/prop_doc_url.asp,"b'<html lang=""en-US"" style=""height: 100%;""><he..."
3,https://www.w3schools.com/jsref/jsref_obj_arra...,"b'<html lang=""en-US"" style=""height: 100%;""><he..."
4,https://www.w3schools.com/jsref/obj_window.asp,"b'<html lang=""en-US"" style=""height: 100%;""><he..."
5,https://www.w3schools.com/jsref/jsref_obj_json...,"b'<html lang=""en-US"" style=""height: 100%;""><he..."
6,https://www.w3schools.com/jsref/jsref_obj_erro...,"b'<html lang=""en-US"" style=""height: 100%;""><he..."


#### Extract titles from HTMLs

In [8]:
import re
title = re.compile(b"<title(?![0-9A-Za-z]).*?>.*?</title>", re.DOTALL)

for row in df.itertuples():  
    m = re.search(title, row.html_bin) 
    print(m.group(0))

b'<title>Index of /hungarian</title>'
b'<title>Index of /C11</title>'
b'<title>HTML DOM URL Property</title>'
b'<title>JavaScript Array Reference</title>'
b'<title>Window Object</title>'
b'<title>JavaScript JSON Reference</title>'
b'<title>JavaScript Error Reference</title>'


In [9]:
# custom wrapper to solve CORS
# def header_modifier(foo):  
#     def wrapper():
#         response = make_response(foo())
#         headers = response.headers
#         headers['Access-Control-Allow-Origin'] = '*' 
#         return response 
#     return wrapper 